# Calcul des cotes à partir du classement (Ligue 1)
Ce notebook montre un algorithme simple pour calculer les **cotes 1N2** d’un match uniquement en utilisant le **classement (points)**.

- Pas de stats avancées
- Bonus domicile ajouté
- Nul fixé à 25%

In [10]:
def calc_prob_from_ranking(home, away, classement, bonus_home=1.1):
    points_home = int(classement.get(home, 0))
    points_away = int(classement.get(away, 0))

    # Bonus domicile
    score_home = points_home * bonus_home
    score_away = points_away

    # ⚠️ Éviter division par zéro
    if score_home + score_away == 0:
        score_home, score_away = 0.5, 0.5

    # Probabilités brutes (sans nul)
    p_home = score_home / (score_home + score_away)
    p_away = score_away / (score_home + score_away)

    # Probabilité du nul (fixe : 25%)
    p_draw = 0.25

    # Réduction home/away pour laisser de la place au nul
    p_home *= (1 - p_draw)
    p_away *= (1 - p_draw)

    return {
        "prob_home": round(p_home, 3),
        "prob_draw": round(p_draw, 3),
        "prob_away": round(p_away, 3),
        "odds_home": round(1/p_home, 2) if p_home > 0 else None,
        "odds_draw": round(1/p_draw, 2),
        "odds_away": round(1/p_away, 2) if p_away > 0 else None
    }


## 📊 Récupération du classement Ligue 1 via API

In [11]:
import requests
import pandas as pd

# URL de l'API pour le classement Ligue 1 2024–2025
url = "https://www.thesportsdb.com/api/v1/json/3/lookuptable.php?l=4334&s=2025-2026"

# Requête à l'API
response = requests.get(url)
r = response.json()

# Vérification de la clé 'table'
if 'table' not in r:
    print("❌ Données de classement introuvables.")
else:
    table = r['table']

    # Construction du DataFrame
    data = pd.DataFrame([{
        "Pos": team["intRank"],
        "Team": team["strTeam"],
        "M": team["intPlayed"],
        "W": team["intWin"],
        "D": team["intDraw"],
        "L": team["intLoss"],
        "G": team["intGoalsFor"],
        "GA": team["intGoalsAgainst"],
        "Diff": team["intGoalDifference"],
        "PTS": team["intPoints"]
    } for team in table])

    # Tri par points décroissants
    data = data.sort_values(by="PTS", ascending=False).reset_index(drop=True)

    # Ajout d'un rang basé sur l'ordre après tri
    data.index = data.index + 1
    data.index.name = "Rang"

    print("🏆 Classement Ligue 1 – 2024/2025 :")
    display(data)


🏆 Classement Ligue 1 – 2024/2025 :


,Pos,Team,M,W,D,L,G,GA,Diff,PTS
Rang,,,,,,,,,,
1,1,Lyon,2,2,0,0,4,0,4,6
2,2,Toulouse,2,2,0,0,3,0,3,6
3,3,Lens,3,2,0,1,5,3,2,6
4,4,Paris SG,2,2,0,0,2,0,2,6
5,5,Strasbourg,2,2,0,0,2,0,2,6
6,6,Lille,2,1,1,0,4,3,1,4
7,13,Rennes,2,1,0,1,1,4,-3,3
8,12,Auxerre,2,1,0,1,2,3,-1,3
9,11,Angers,2,1,0,1,1,1,0,3


In [12]:
# ---- Conversion du DataFrame en dictionnaire {Equipe: Points} ----
classement_pts = data.set_index("Team")["PTS"].astype(int).to_dict()

# Vérification
classement_pts

{'Lyon': 6,
 'Toulouse': 6,
 'Lens': 6,
 'Paris SG': 6,
 'Strasbourg': 6,
 'Lille': 4,
 'Rennes': 3,
 'Auxerre': 3,
 'Angers': 3,
 'Nice': 3,
 'Monaco': 3,
 'Marseille': 3,
 'Lorient': 3,
 'Brest': 1,
 'Nantes': 0,
 'Le Havre': 0,
 'Paris FC': 0,
 'Metz': 0}

In [13]:
# Calcul des cotes pour PSG vs Marseille
cotes = calc_prob_from_ranking("Nantes", "Auxerre", classement_pts)
cotes


{'prob_home': 0.0,
 'prob_draw': 0.25,
 'prob_away': 0.75,
 'odds_home': None,
 'odds_draw': 4.0,
 'odds_away': 1.33}